In [2]:
#Steven Zajac-Descôteaux

In [1]:
import scipy.optimize as opt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sympy as sy
import seaborn as sns
import scipy as sp
import statsmodels.api as sm
import statsmodels.stats.api as sms

# 1. Bisection


One of the most common algorithms for numerical root-finding is *bisection*.

To understand the idea, recall the well-known game where:

- Player A thinks of a secret number between 1 and 100  
- Player B asks if it’s less than 50  
  
  - If yes, B asks if it’s less than 25  
  - If no, B asks if it’s less than 75  
  

And so on.

This is bisection, a relative of [binary search](https://en.wikipedia.org/wiki/Binary_search_algorithm). It works for all sufficiently well behaved increasing continuous functions with $ f(a) < 0 < f(b) $. 

Write an implementation of the bisection algorith, `bisect(f, lower, upper, tol)` which, given a function `f`, a lower bound `lower` and an upper bound `upper` finds the point `x` where `f(x) = 0`. The parameter `tol` is a numerical tolerance, you should stop once your step size is smaller than `tol`.


Use it to minimize the function:

$$
f(x) = \sin(4 (x - 1/4)) + x + x^{20} - 1 \tag{2}
$$

in python: `lambda x: np.sin(4 * (x - 1/4)) + x + x**20 - 1`

The value where f(x) = 0 should be around `0.408`

In [2]:
#https://www.math.ubc.ca/~pwalls/math-python/roots-optimization/bisection/

def bisect(f,lower,upper,tol):
    
    """
    f is given function
    lower is lower bound
    upper is upper bound
    tol is numerical tolerance
    
    Approximate solution of f(x)=0 on interval [a,b] by bisection method
    
    """
    
    if f(lower)*f(upper) >= 0:
        print('Bisection method fails.')
        return None
    
    a = lower
    b = upper
    
    for n in range(1,tol+1): #number of iterations or tol chosen. (run this until tol is reached)
        mid = (a + b)/2 #midpoint
        f_mid = f(mid)
        
        #Determine the next subinterval
        if f(a) * f_mid < 0: #[a1,b1]: a1=a0 and b1=mid0
            a = a
            b = mid
        elif f(b) * f_mid < 0: #[a1,b1]: a1=mid0 and b1=b0
            a = mid
            b = b
        elif f_mid == 0: #exact solution if f_mid=0
            print('Exact Solution')
            return mid
        else:
            print('Bisection method fails.')
            return None
    
    return (a + b)/2 #return the midpoint value at tol or N mN = (aN + bN)/2


f = lambda x: np.sin(4 * (x - 1/4)) + x + x**20 - 1
lower = 0
upper = 1
tol = 1000

bisect(f,lower,upper,tol)

Exact Solution


0.40829350427936706

# 1.2 (stretch) Recursive Bisect

Write a recursive version of the bisection algorithm

In [ ]:
def bisect_rec(f, lower, upper, tol):
    

# 2.1 Movies Regression

Write the best linear regression model you can on the [Movies Dataset](https://www.kaggle.com/rounakbanik/the-movies-dataset?select=ratings.csv) to predict the profitability of a movie (revenue - budget). Maintain the interpretability of the model.

Few notes:

1. Clean your data! Movies where the budget or revenue are invalid should be thrown out

2. Be creative with feature engineering. You can include processing to one-hot encode the type of movie, etc.

3. The model should be useful for someone **who is thinking about making a movie**. So features like the popularity can't be used. You could, however, use the ratings to figure out if making "good" or "oscar bait" movies is a profitable strategy.

In [3]:
movies_url = {
"movies_metadata": "1RLvh6rhzYiDDjPaudDgyS9LmqjbKH-wh",
"keywords": "1YLOIxb-EPC_7QpkmRqkq9E6j7iqmoEh3",
"ratings": "1_5HNurSOMnU0JIcXBJ5mv1NaXCx9oCVG",
"credits": "1bX9othXfLu5NZbVZtIPGV5Hbn8b5URPf",
"ratings_small": "1fCWT69efrj4Oxdm8ZNoTeSahCOy6_u6w",
"links_small": "1fh6pS7XuNgnZk2J3EmYk_9jO_Au_6C15",
"links": "1hWUSMo_GwkfmhehKqs8Rs6mWIauklkbP",
}

def read_gdrive(url):
    """
    Reads file from Google Drive sharing link
    """
    path = 'https://drive.google.com/uc?export=download&id='+url
    return pd.read_csv(path)

In [4]:
movies_metadata = read_gdrive('1RLvh6rhzYiDDjPaudDgyS9LmqjbKH-wh')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [5]:
#For later
def recur_elim(model, X, pval=0.05): 
    """
    Use recursion to eliminate p-values that are above 0.05 at least.
    Or as chosen.
    """
    global keep #Need this to be able to use it later? (I think)
    
    new_pval= pd.DataFrame(model.pvalues) #Put pvalues into df
    new_pval.columns = ['pvalue'] #rename col
    
    coefs = pd.DataFrame(model.params) #Put coef names
    coefs.columns = ['coefs'] #name col

    #Did recursion first and base case last 
    if len(new_pval[new_pval['pvalue'] > pval].index) > 0:
        #Drop where pvalue is greater than input limit of p-val
        drop_Pval = (new_pval[new_pval['pvalue'] > pval].index)
        X_dropped = X.drop(columns=drop_Pval) #New X without high p-vals
        model = sm.OLS(y, X_dropped).fit(cov_type='HC2')
        
        return recur_elim(model, X_dropped)
    
    #Base case: no mre p-vals > input p-val
    #Length of new_pval list where pvals are larger than input
    else:
        keep = list(X.columns.values)
    return keep

In [6]:
movies = movies_metadata.copy()

movies['belongs_to_collection'] = movies['belongs_to_collection'].notna().astype(int)

#Drop columns that are irrelevant, have too much missing or are incorrect ie. status. 
movies.drop(['homepage','id','imdb_id','overview','tagline','poster_path','original_title',
            'video','spoken_languages','production_companies','vote_count','vote_average','popularity',
            'adult','status','runtime'],axis=1,inplace=True)

movies.dropna(inplace=True)

movies.budget = movies.budget.astype(float)

#keep only where values are not 0
movies = movies[movies['budget'] != 0]
movies = movies[movies['revenue'] != 0 ]

#Make a profit col 
movies['profit'] = movies['revenue'].subtract(movies['budget'])

#col = ['spoken_languages','production_companies','production_countries','genres']

#for c in col:
#    movies[c] = movies[c].fillna(np.nan).apply()
#    movies[c] = movies[c].apply(lambda x: [i['name']for i in x])

movies['release_date'] = pd.to_datetime(movies['release_date'])

#Convert release date to season category using dictionary and mapping
seasons = ['win','win','spr','spr','spr','sum','sum','sum','fall','fall','fall','win']
month_season = dict(zip(range(1,13),seasons))

#movies.release_date = movies.release_date.dt.month.map(month_season) --> added below to remove a step
movies[['spr_rel','sum_rel','win_rel']] = pd.get_dummies(movies.release_date.dt.month.map(month_season),
                                                         drop_first=True)

#Get first genre. Assuming like an ingredient list on packaging, first genre is most pertinent 
def get_genre(data):
    for g in eval(data):
        return g['name']

def get_countries(data):
    for c in eval(data):
        return c['name']

#Convert genre1 as category.......need to make sure maybe OHE
movies['genre'] = (movies.genres.apply(get_genre)).astype('category')
movies['country'] = (movies.production_countries.apply(get_countries)).astype('category')
movies.dropna(inplace=True)

#If english vs other language. 
movies['lang_en'] = (movies.original_language=='en').astype(int)

movies.drop(['revenue','genres','release_date','title','original_language',
             'production_countries'],axis=1,inplace=True)

movies = pd.get_dummies(movies,drop_first=True) #Convert Genre col in dummies

movies['const'] = 1

In [7]:
X = movies.drop('profit',axis=1)
y = movies['profit']

model = sm.OLS(y,X).fit(cov_type='HC2')

recur_elim(model,X) #use recursion to drop p-values that are above 0.05

X = movies[keep]
y = movies.profit 

est = sm.OLS(y,X).fit(cov_type='HC2')

est.summary()

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:1830: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 31, but rank is 30
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 profit   R-squared:                       0.406
Model:                            OLS   Adj. R-squared:                  0.403
Method:                 Least Squares   F-statistic:                 4.235e+04
Date:                Wed, 03 Feb 2021   Prob (F-statistic):               0.00
Time:                        15:40:10   Log-Likelihood:            -1.0582e+05
No. Observations:                5313   AIC:                         2.117e+05
Df Residuals:                    5281   BIC:                         2.119e+05
Df Model:                          31                                         
Covariance Type:                  HC2                                         
====================================================================================================
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
belongs_to_collection              8.22e+07   4.53e+06     18.153      0.000    7.33e+07    9.11e+07
budget                               1.8536      0.104     17.814      0.000       1.650       2.058
genre_Animation                    4.74e+07   1.64e+07      2.888      0.004    1.52e+07    7.96e+07
genre_Comedy                      1.195e+07   2.98e+06      4.009      0.000    6.11e+06    1.78e+07
genre_Documentary                 2.372e+07   6.52e+06      3.638      0.000    1.09e+07    3.65e+07
genre_Drama                       1.363e+07   3.25e+06      4.192      0.000    7.26e+06       2e+07
genre_Foreign                     1.158e+07   4.84e+06      2.393      0.017     2.1e+06    2.11e+07
genre_History                     3.127e+07   1.09e+07      2.878      0.004    9.98e+06    5.26e+07
genre_Romance                     2.843e+07   8.28e+06      3.432      0.001    1.22e+07    4.47e+07
genre_TV Movie                    7.094e+07   3.78e+06     18.780      0.000    6.35e+07    7.83e+07
country_Bahamas                   6.857e+07   1.83e+07      3.740      0.000    3.26e+07    1.05e+08
country_Burkina Faso              2.941e+07   3.59e+06      8.200      0.000    2.24e+07    3.64e+07
country_Chile                     2.806e+07   6.54e+06      4.289      0.000    1.52e+07    4.09e+07
country_Czech Republic           -6.296e+07   2.94e+07     -2.141      0.032   -1.21e+08   -5.32e+06
country_India                     3.427e+07   4.81e+06      7.121      0.000    2.48e+07    4.37e+07
country_Indonesia                -4.804e+07   7.08e+06     -6.784      0.000   -6.19e+07   -3.42e+07
country_Iran                      2.996e+07   3.63e+06      8.253      0.000    2.28e+07    3.71e+07
country_Malaysia                  6.384e+07   3.07e+06     20.817      0.000    5.78e+07    6.99e+07
country_Mali                      2.398e+07   3.54e+06      6.780      0.000     1.7e+07    3.09e+07
country_Malta                    -3.006e+07   3.68e+06     -8.167      0.000   -3.73e+07   -2.28e+07
country_New Zealand               1.373e+08   5.46e+07      2.517      0.012    3.04e+07    2.44e+08
country_Peru                      8.516e+07   3.39e+06     25.150      0.000    7.85e+07    9.18e+07
country_Philippines               3.838e+07   6.25e+06      6.144      0.000    2.61e+07    5.06e+07
country_Qatar                     2.611e+07   3.57e+06      7.312      0.000    1.91e+07    3.31e+07
country_Serbia                    3.827e+07   4.78e+06      8.010      0.000    2.89e+07    4.76e+07
country_South Korea               3.379e+07   9.39e+06      3.598      0.000    1.54e+07    5.22e+07
country_Sweden                    2.342e+07   6.43e+06      3.641      0.000    1.08e+07     3.6e+07
country_Taiwan                     3.28e+07   4.97e+0

# 2.2 Movies Manual Regression

Use your `X` and `y` matrix from 2.1 to calculate the linear regression yourself using the normal equation $(X^T X)^{-1}X^Ty$.

Verify that the coefficients are the same.

In [8]:
#https://towardsdatascience.com/performing-linear-regression-using-the-normal-equation-6372ed3c57

β = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y) # normal equation
#Need to use linalg from Numpy
β.reshape(32,1)

array([[ 8.21960906e+07],
       [ 1.85358989e+00],
       [ 4.73971514e+07],
       [ 1.19481705e+07],
       [ 2.37202919e+07],
       [ 1.36322483e+07],
       [ 1.15835657e+07],
       [ 3.12662299e+07],
       [ 2.84297369e+07],
       [ 7.09365012e+07],
       [ 6.85715031e+07],
       [ 2.94105115e+07],
       [ 2.80619933e+07],
       [-6.29590579e+07],
       [ 3.42746994e+07],
       [-4.80436078e+07],
       [ 2.99585562e+07],
       [ 6.38423197e+07],
       [ 2.39760226e+07],
       [-3.00636738e+07],
       [ 1.37334446e+08],
       [ 8.51554876e+07],
       [ 3.83828452e+07],
       [ 2.61125355e+07],
       [ 3.82663554e+07],
       [ 3.37924868e+07],
       [ 2.34233922e+07],
       [ 3.28016577e+07],
       [ 3.61886423e+07],
       [ 2.11816222e+07],
       [-1.78235675e+08],
       [-4.32044506e+07]])

In [9]:
est.params

belongs_to_collection               8.219609e+07
budget                              1.853590e+00
genre_Animation                     4.739715e+07
genre_Comedy                        1.194817e+07
genre_Documentary                   2.372029e+07
genre_Drama                         1.363225e+07
genre_Foreign                       1.158357e+07
genre_History                       3.126623e+07
genre_Romance                       2.842974e+07
genre_TV Movie                      7.093650e+07
country_Bahamas                     6.857150e+07
country_Burkina Faso                2.941051e+07
country_Chile                       2.806199e+07
country_Czech Republic             -6.295906e+07
country_India                       3.427470e+07
country_Indonesia                  -4.804361e+07
country_Iran                        2.995856e+07
country_Malaysia                    6.384232e+07
country_Mali                        2.397602e+07
country_Malta                      -3.006367e+07
country_New Zealand 

# 2.3 Movies gradient descent regression

Use your `X` and `y` matrix from 2.1 to calculate the linear regression yourself using **gradient descent**. 

Hint: use `scipy.optimize` and remember we're finding the $\beta$ that minimizes the squared loss function of linear regression: $f(\beta) = (\beta X - y)^2$. This will look like part 3 of this lecture.

Verify your coefficients are similar to the ones in 2.1 and 2.2. They won't necessarily be exactly the same, but should be roughly similar.

In [10]:
def gr_dscnt(bhat,y,X):
    return np.sum( ((X.dot(bhat)) - y)**2)

In [21]:
from scipy.optimize import minimize

X = movies[keep] #Constant already added 
y = movies.profit

#create beta hat vector to maximize on
#will store the values of maximum likelihood beta parameters
#Set to rand rather than 0
bhat = np.random.rand(np.shape(X)[1])

#unvectorized MLE estimation
gradient_est = minimize(gr_dscnt, bhat, args=(y,X), method='Powell')

#print vector of maximized betahats
gradient_est['x']

array([ 8.20073548e+07,  1.85336686e+00,  4.71574627e+07,  1.16399204e+07,
        2.34212450e+07,  1.33664499e+07,  1.13060534e+07,  3.10368737e+07,
        2.81658291e+07,  7.09325291e+07,  6.86613761e+07,  2.96714283e+07,
        2.82351189e+07, -6.28205729e+07,  3.44394187e+07, -4.78569661e+07,
        3.02193399e+07,  6.38412139e+07,  2.42369328e+07, -3.00588254e+07,
        1.37485838e+08,  8.51603921e+07,  3.85510865e+07,  2.63735075e+07,
        3.83367040e+07,  3.38819912e+07,  2.35589589e+07,  3.31049688e+07,
        3.63733993e+07,  2.16180605e+07, -1.78207015e+08, -4.32034448e+07])

In [22]:
np.random.rand()

0.3983247565365491